## k-fold validation

In [1]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
import numpy as np

### Model configuration

In [11]:
batch_size = 50
img_width, img_height, img_num_channels = 32, 32, 3
loss_function = sparse_categorical_crossentropy
no_classes = 100
no_epochs = 5
optimizer = Adam()
verbosity = 1
num_folds = 5

### Load CIFAR-10 data

In [12]:
(input_train, target_train), (input_test, target_test) = cifar10.load_data()

### Determine shape of the data

In [13]:
input_shape = (img_width, img_height, img_num_channels)

### Parse numbers as floats

In [14]:
input_train = input_train.astype('float32')
input_test = input_test.astype('float32')

### Normalize data

In [16]:
input_train = input_train / 255
input_test = input_test / 255

### Define per-fold score containers

In [17]:
acc_per_fold = []
loss_per_fold = []

### Merge inputs and targets

In [18]:
inputs = np.concatenate((input_train, input_test), axis=0)
targets = np.concatenate((target_train, target_test), axis=0)

### Define the K-fold Cross Validator

In [19]:
kfold = KFold(n_splits=num_folds, shuffle=True)

### K-fold Cross Validation model evaluation

In [20]:
fold_no = 1
for train, test in kfold.split(inputs, targets):

  # Define the model architecture
  model = Sequential()
  model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Flatten())
  model.add(Dense(256, activation='relu'))
  model.add(Dense(128, activation='relu'))
  model.add(Dense(no_classes, activation='softmax'))

  # Compile the model
  model.compile(loss=loss_function,
                optimizer=optimizer,
                metrics=['accuracy'])


  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  # Fit data to model
  history = model.fit(inputs[train], targets[train],
              batch_size=batch_size,
              epochs=no_epochs,
              verbose=verbosity)

  # Generate generalization metrics
  scores = model.evaluate(inputs[test], targets[test], verbose=0)
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])

  # Increase fold number
  fold_no = fold_no + 1

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/5
960/960 [==============================] - 56s 56ms/step - loss: 2.2946 - accuracy: 0.1327
Epoch 2/5
960/960 [==============================] - 54s 56ms/step - loss: 1.9281 - accuracy: 0.3063
Epoch 3/5
960/960 [==============================] - 52s 54ms/step - loss: 1.7352 - accuracy: 0.3752
Epoch 4/5
960/960 [==============================] - 49s 51ms/step - loss: 1.6148 - accuracy: 0.4217
Epoch 5/5
960/960 [==============================] - 51s 53ms/step - loss: 1.5258 - accuracy: 0.4556
Score for fold 1: loss of 1.4955583810806274; accuracy of 46.47499918937683%
------------------------------------------------------------------------
Training for fold 2 ...
Epoch 1/5
960/960 [==============================] - 50s 51ms/step - loss: 2.3300 - accuracy: 0.1007
Epoch 2/5
960/960 [==============================] - 48s 50ms/step - loss: 2.3076 - accuracy: 0.1003
Epoch 3/5
960/960 [===

In [21]:
# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 1.4955583810806274 - Accuracy: 46.47499918937683%
------------------------------------------------------------------------
> Fold 2 - Loss: 2.3042306900024414 - Accuracy: 10.16666665673256%
------------------------------------------------------------------------
> Fold 3 - Loss: 1.734125018119812 - Accuracy: 37.033334374427795%
------------------------------------------------------------------------
> Fold 4 - Loss: 1.7199279069900513 - Accuracy: 38.56666684150696%
------------------------------------------------------------------------
> Fold 5 - Loss: 2.3041908740997314 - Accuracy: 9.866666793823242%
------------------------------------------------------------------------
Average scores for all folds:
> Accuracy: 28.421666771173477 (+- 15.365791291540202)
> Loss: 1.9116065740585326
---------------------------